In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.merrill import Merrill
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
import pandas_datareader as pdr

In [2]:
## Loading Constants
end = datetime.now().strftime("%Y-%m-%d")
start = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("portfolio")
market = Market()
merrill = Merrill()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
reload = False
model_range = range(len(tickers["Symbol"]))
year = datetime.now().year
quarter = math.ceil((datetime.now().month - 1) // 3) + 1
datasets = ["pdr"
#             ,"tiingo"
#             ,"finnhub"
           ]

In [3]:
# sims = []
# training_days = 100
# gap = 5
# week_gap = int(gap/5)
# training_years = 4
# training_days = 100
# daily_gap = 2
year = datetime.now().year
week = datetime.now().isocalendar()[1]
sec.connect()
strat_db.connect()
for dataset in tqdm(datasets):
    classification = strat_db.retrieve_data("dataset_{}_week_classification".format(dataset))
    regression = strat_db.retrieve_data("dataset_{}_week_regression".format(dataset))
    models = strat_db.retrieve_data("{}_weekly_models".format(dataset))
    features = strat_db.retrieve_data("{}_weekly_model_features".format(dataset))
    strat_db.drop_table("{}_weekly_recs".format(dataset))
    for i in tqdm(model_range):
        try:
            ticker = tickers.iloc[i]["Symbol"].replace(".","-")
            price_results = models[models["ticker"]==ticker]
            ticker_features = features[features["ticker"]==ticker]
            mr = ModelPreprocessor(ticker)
            prot = ProductTransformer(ticker,start,end)
            product_qpds = []
            for i in range(len(price_results)):
                price_result = price_results.iloc[i]
                if price_result["model_type"] == "regression":
                    weekly_price_data = regression.copy()
                    product_qpd = mr.day_trade_preprocess_regression_rec(weekly_price_data.copy(),ticker)
                else:
                    weekly_price_data = classification.copy()
                    product_qpd = mr.day_trade_preprocess_classify_rec(weekly_price_data.copy(),ticker)
                product_qpds.append(product_qpd)
                product_week = week
            sim = prot.merge_weeklies_calcs(product_qpds,price_results,year,product_week,"price",ticker_features)
            if len(sim) > 0:
                strat_db.store_data("{}_weekly_recs".format(dataset),pd.DataFrame([sim]))
        except Exception as e:
            message = {"status":"weekly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
            print(message)
sec.close()
strat_db.close()

  7%|████████████▋                                                                                                                                                                      | 36/506 [00:05<01:14,  6.30it/s]


 14%|█████████████████████████▊                                                                                                                                                         | 73/506 [00:11<01:16,  5.69it/s]


 22%|██████████████████████████████████████▋                                                                                                                                           | 110/506 [00:16<01:02,  6.36it/s]


 29%|████████████████████████████████████████████████████                                                                                                                              | 148/506 [00:22<01:01,  5.83it/s]


 37%|█████████████████████████████████████████████████████████████████                                                                                                                 | 185/506 [00:29<00:47,  6.70it/s]


...

 66%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                            | 333/506 [00:51<00:28,  6.08it/s]


 73%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                               | 370/506 [00:57<00:20,  6.50it/s]


 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                  | 407/506 [01:02<00:15,  6.51it/s]


 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏               | 461/506 [01:09<00:04, 10.19it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:17<00:00, 77.45s/it]


In [4]:
sim

{'weekly_price_classification_prediction': 1.0,
 'weekly_price_classification_score': 0.5714285714285714,
 'weekly_price_regression_prediction': 164.58585,
 'weekly_price_regression_score': 0.028994788863883614,
 'week': 16,
 'year': 2021,
 'ticker': 'ZTS'}